## Imports

In [48]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_columns', 50)
SEED = 42

## Data I/O

Please enter your exact data path

In [49]:
from pathlib import Path
DATA_DIR = Path.cwd().parents[2] / "data"



df = pd.read_csv(f"{DATA_DIR}/corn_climate_risk_futures_daily_master.csv", parse_dates=["date_on"])
shares = pd.read_csv(f"{DATA_DIR}/corn_regional_market_share.csv")

df.shape, df["date_on"].min(), df["date_on"].max()

((42180, 42),
 Timestamp('2024-01-05 00:00:00'),
 Timestamp('2025-11-21 00:00:00'))

In [50]:
df

,ID,crop_name,country_name,country_code,region_name,region_id,harvest_period,growing_season_year,date_on,climate_risk_cnt_locations_heat_stress_risk_low,climate_risk_cnt_locations_heat_stress_risk_medium,climate_risk_cnt_locations_heat_stress_risk_high,climate_risk_cnt_locations_unseasonably_cold_risk_low,climate_risk_cnt_locations_unseasonably_cold_risk_medium,climate_risk_cnt_locations_unseasonably_cold_risk_high,climate_risk_cnt_locations_excess_precip_risk_low,climate_risk_cnt_locations_excess_precip_risk_medium,climate_risk_cnt_locations_excess_precip_risk_high,climate_risk_cnt_locations_drought_risk_low,climate_risk_cnt_locations_drought_risk_medium,climate_risk_cnt_locations_drought_risk_high,futures_close_ZC_1,futures_close_ZC_2,futures_close_ZW_1,futures_close_ZS_1,futures_zc1_ret_pct,futures_zc1_ret_log,futures_zc_term_spread,futures_zc_term_ratio,futures_zc1_ma_20,futures_zc1_ma_60,futures_zc1_ma_120,futures_zc1_vol_20,futures_zc1_vol_60,futures_zw_zc_spread,futures_zc_zw_ratio,futures_zs_zc_spread,futures_zc_zs_ratio,date_on_year,date_on_month,date_on_year_month,Usage
0,171ac519-e661-4812-ab64-6266586a2a48,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2024,2024-04-01,23,0,0,23,0,0,23,0,0,2,21,0,435.50,449.25,557.00,1185.75,-0.014706,-0.014815,13.75,1.031573,431.2250,434.400000,454.350000,0.012349,0.010953,121.50,0.781867,750.25,0.367278,2024,4,2024_04,Private
1,5bb5c8ba-1a30-4eb8-8608-4755c1ebcf44,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2024,2024-04-02,23,0,0,23,0,0,23,0,0,2,19,2,426.50,441.00,545.25,1174.00,-0.020666,-0.020882,14.50,1.033998,431.6750,433.733333,453.835417,0.013169,0.011230,118.75,0.782210,747.50,0.363288,2024,4,2024_04,Private
2,9da691be-3acc-49c7-9bb1-ec0ccc341e58,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2024,2024-04-03,23,0,0,23,0,0,23,0,0,2,20,1,431.75,445.00,556.00,1182.25,0.012309,0.012234,13.25,1.030689,432.5875,433.250000,453.387500,0.013140,0.011275,124.25,0.776529,750.50,0.365193,2024,4,2024_04,Private
3,2c97667b-0520-4fca-a5ea-9d3de05fa269,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2024,2024-04-04,23,0,0,23,0,0,23,0,0,3,18,2,435.25,447.50,556.25,1180.00,0.008107,0.008074,12.25,1.028145,433.4500,432.920833,452.947917,0.013058,0.011233,121.00,0.782472,744.75,0.368856,2024,4,2024_04,Private
4,d5c60fa8-ba3c-4ee4-aee8-0ac77e103a72,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2024,2024-04-05,23,0,0,23,0,0,23,0,0,2,19,2,434.25,446.75,567.25,1185.00,-0.002298,-0.002300,12.50,1.028785,433.8625,432.504167,452.433333,0.012452,0.011158,133.00,0.765535,750.75,0.366456,2024,4,2024_04,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42175,67227dad-15db-4276-bb58-8a3eb1825dba,Corn: Commodity Tracked,United States,US,Wisconsin,216ae69a-e4e3-4319-a408-54b9fb807570,Harvest,2026,2025-11-18,7,0,0,7,0,0,7,0,0,3,4,0,436.75,449.50,546.50,1153.50,0.004600,0.004590,12.75,1.029193,431.4000,419.591667,413.779167,0.010429,0.014111,109.75,0.799177,716.75,0.378630,2025,11,2025_11,Private
42176,438c1337-14f2-4c89-9782-8836a603fa95,Corn: Commodity Tracked,United States,US,Wisconsin,216ae69a-e4e3-4319-a408-54b9fb807570,Harvest,2026,2025-11-19,7,0,0,7,0,0,7,0,0,3,4,0,429.75,441.50,536.75,1136.25,-0.016027,-0.016157,11.75,1.027341,431.7375,420.295833,413.660417,0.011079,0.014279,107.00,0.800652,706.50,0.378218,2025,11,2025_11,Private
42177,cc8988b0-51af-4184-a956-5f9933f92aa8,Corn: Commodity Tracked,United States,US,Wisconsin,216ae69a-e4e3-4319-a408-54b9fb807570,Harvest,2026,2025-11-20,7,0,0,7,0,0,7,0,0,2,5,0,426.50,437.75,527.00,1122.50,-0.007563,-0.007591,11.25,1.026377,431.6625,421.029167,413.562500,0.010915,0.014202,100.50,0.809298,696

## Configuration

In [51]:
RISKS = [
    "heat_stress",
    "unseasonably_cold",
    "excess_precip",
    "drought",
]

BASE_COLS = [
    "ID",
    "date_on",
    "crop_name",
    "country_name",
    "region_name",
]

# we are not using futures columns during feature engineering. This definition is just for submission format.
FUTURES_COLS = [
    c for c in df.columns
    if c.startswith("futures_")
]

ROLL_WINDOWS = [400, 410, 420, 430] # long term patterns

## Feature Engineering

In [52]:
def sort_and_reset(df: pd.DataFrame, by: list[str]) -> pd.DataFrame:
    return df.sort_values(by, kind='mergesort').reset_index(drop=True)

def add_production_share(df: pd.DataFrame, shares: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.merge(
        shares[["region_id", "percent_country_production"]],
        on="region_id",
        how="left",
    )
    df["percent_country_production"] = df["percent_country_production"].fillna(1.0)
    return df

def compute_risk_score(df: pd.DataFrame, risk: str) -> pd.Series:
    low = f"climate_risk_cnt_locations_{risk}_risk_low"
    med = f"climate_risk_cnt_locations_{risk}_risk_medium"
    high = f"climate_risk_cnt_locations_{risk}_risk_high"
    total = df[[low, med, high]].sum(axis=1)
    return (df[med] + 2 * df[high]) / (total + 1e-6)

def add_risk_features(df: pd.DataFrame, risks: list[str]) -> pd.DataFrame:
    df = df.copy()
    for r in risks:
        score = compute_risk_score(df, r)
        df[f"climate_risk_{r}_score"] = score
        df[f"climate_risk_{r}_weighted"] = (score * df["percent_country_production"] / 100)
    return df

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    dt = df["date_on"]
    df["day_of_year"] = dt.dt.dayofyear
    df["quarter"] = dt.dt.quarter
    return df

def add_composites(df: pd.DataFrame, risks: list[str]) -> pd.DataFrame:
    df = df.copy()
    df["climate_risk_temperature_stress"] = df[
        ["climate_risk_heat_stress_score", "climate_risk_unseasonably_cold_score"]
    ].max(axis=1)
    df["climate_risk_precipitation_stress"] = df[
        ["climate_risk_excess_precip_score", "climate_risk_drought_score"]
    ].max(axis=1)
    cols = [f"climate_risk_{r}_score" for r in risks]
    df["climate_risk_overall_stress"] = df[cols].max(axis=1)
    df["climate_risk_combined_stress"] = df[cols].mean(axis=1)
    return df

def add_rolling_features(df: pd.DataFrame, risks: list[str], windows: list[int]) -> pd.DataFrame:
    df = df.copy()
    df['_original_order'] = range(len(df))
    df = sort_and_reset(df, ["region_id", "date_on"])
    
    for w in windows:
        for r in risks:
            col = f"climate_risk_{r}_score"
            grp = df.groupby("region_id", sort=False)[col]
            
            df[f"{col}_ma_{w}d"] = (
                grp.rolling(w, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)
            )
            
            df[f"{col}_max_{w}d"] = (
                grp.rolling(w, min_periods=1)
                .max()
                .reset_index(level=0, drop=True)
            )
    
    df = df.sort_values('_original_order').reset_index(drop=True)
    df = df.drop(columns='_original_order')
    return df

def add_momentum_features(df: pd.DataFrame, risks: list[str]) -> pd.DataFrame:
    df = df.copy()
    df['_original_order'] = range(len(df))
    df = sort_and_reset(df, ["region_id", "date_on"])
    
    for r in risks:
        col = f"climate_risk_{r}_score"
        grp = df.groupby("region_id", sort=False)[col]
        
        df[f"{col}_change_1d"] = grp.diff(1)
        df[f"{col}_change_7d"] = grp.diff(7)
        df[f"{col}_acceleration"] = (
            df.groupby("region_id", sort=False)[f"{col}_change_1d"].diff()
        )
    
    df = df.sort_values('_original_order').reset_index(drop=True)
    df = df.drop(columns='_original_order')
    return df

def add_country_aggregates(df: pd.DataFrame, risks: list[str]) -> pd.DataFrame:
    df = df.copy()
    
    for r in risks:
        score = f"climate_risk_{r}_score"
        weight = f"climate_risk_{r}_weighted"
        
        agg = (
            df.groupby(["country_name", "date_on"], sort=False)
            .agg(
                **{
                    f"climate_risk_country_{r}_mean": (score, "mean"),
                    f"climate_risk_country_{r}_max": (score, "max"),
                    f"climate_risk_country_{r}_std": (score, "std"),
                    f"climate_risk_country_{r}_weighted_sum": (weight, "sum"),
                }
            )
            .reset_index()
            .round(4)
        )
        
        df = df.merge(agg, on=["country_name", "date_on"], how="left")
    
    return df

def add_country_weighted_rollings(df: pd.DataFrame, risks: list[str], windows: list[int]) -> pd.DataFrame:
    df = df.copy()
    df['_original_order'] = range(len(df))
    df = sort_and_reset(df, ["country_name", "date_on"])
    
    for r in risks:
        col = f"climate_risk_{r}_weighted"
        grp = df.groupby("country_name", sort=False)[col]
        
        for w in windows:
            df[f"{col}_country_sum_ma_{w}d"] = (
                grp.rolling(w, min_periods=1)
                .sum()
                .reset_index(level=0, drop=True)
            )
            
            df[f"{col}_country_max_ma_{w}d"] = (
                grp.rolling(w, min_periods=1)
                .max()
                .reset_index(level=0, drop=True)
            )
    
    df = df.sort_values('_original_order').reset_index(drop=True)
    df = df.drop(columns='_original_order')
    return df

def add_country_weighted_cumsum(df: pd.DataFrame, risks: list[str]) -> pd.DataFrame:
    df = df.copy()
    df = sort_and_reset(df, ["country_name", "date_on"])

    for r in risks:
        w = f"climate_risk_{r}_weighted"
        df[f"{w}_country_cumsum"] = (
            df.groupby("country_name", sort=False)[w].cumsum()
        )

    return df

In [53]:
df_engineered = (
    df
    .pipe(add_time_features)
    .pipe(add_production_share, shares)
    .pipe(add_risk_features, RISKS)
    .pipe(add_composites, RISKS)
    .pipe(add_rolling_features, RISKS, ROLL_WINDOWS)
    .pipe(add_momentum_features, RISKS)
    .pipe(add_country_aggregates, RISKS)
    .pipe(add_country_weighted_rollings, RISKS, ROLL_WINDOWS)
    .pipe(add_country_weighted_cumsum, ["drought"])
)

In [54]:
from scipy import stats

def compute_monthly_climate_futures_correlations(
    df: pd.DataFrame,
) -> pd.DataFrame:
    climate_cols = [c for c in df.columns if c.startswith("climate_risk_")]
    futures_cols = [c for c in df.columns if c.startswith("futures_")]

    df = df.copy()
    df["month"] = df["date_on"].dt.month

    out = []

    for (crop, country, month), g in df.groupby(
        ["crop_name", "country_name", "month"]
    ):
        x = g[climate_cols].to_numpy()
        y = g[futures_cols].to_numpy()

        x_std = x.std(axis=0)
        y_std = y.std(axis=0)

        x_mask = x_std > 0
        y_mask = y_std > 0

        if not (x_mask.any() and y_mask.any()):
            continue

        x = x[:, x_mask]
        y = y[:, y_mask]

        x_cols = np.array(climate_cols)[x_mask]
        y_cols = np.array(futures_cols)[y_mask]

        x = x - x.mean(axis=0)
        y = y - y.mean(axis=0)

        denom_x = np.sqrt((x ** 2).sum(axis=0))
        denom_y = np.sqrt((y ** 2).sum(axis=0))

        corr = (x.T @ y) / np.outer(denom_x, denom_y)

        for i, cx in enumerate(x_cols):
            for j, fy in enumerate(y_cols):
                out.append({
                    "crop_name": crop,
                    "country_name": country,
                    "month": month,
                    "climate_variable": cx,
                    "futures_variable": fy,
                    "correlation": round(float(corr[i, j]), 5),
                })

    return pd.DataFrame(out)

def calculate_cfcs_from_arrays(
    abs_corr: np.ndarray,
    sig_thr: float,
) -> dict:
    if abs_corr.size == 0:
        return {
            "cfcs_score": 0.0,
            "avg_significant_correlation": 0.0,
            "max_abs_correlation": 0.0,
            "significant_correlations_pct": 0.0,
            "total_correlations": 0,
            "significant_correlations": 0,
        }

    sig = abs_corr >= sig_thr

    avg_sig = abs_corr[sig].mean() if sig.any() else 0.0
    max_corr = abs_corr.max()
    sig_pct = sig.mean()

    cfcs = (
        0.5 * min(100.0, avg_sig * 100)
        + 0.3 * min(100.0, max_corr * 100)
        + 0.2 * (sig_pct * 100)
    )

    return {
        "cfcs_score": round(cfcs, 2),
        "avg_significant_correlation": round(avg_sig, 4),
        "max_abs_correlation": round(max_corr, 4),
        "significant_correlations_pct": round(sig_pct * 100, 2),
        "total_correlations": int(abs_corr.size),
        "significant_correlations": int(sig.sum()),
    }

def calculate_cfcs_score(
    correlations_df: pd.DataFrame,
    sig_thr: float = 0.5,
) -> dict:
    abs_corr = correlations_df["correlation"].dropna().abs().to_numpy()
    return calculate_cfcs_from_arrays(abs_corr, sig_thr)


## Holdout Validation

In [55]:
# only last year as a holdout validation set
df_2025 = df_engineered.dropna()

monthly_corr_2025 = compute_monthly_climate_futures_correlations(df_2025)

score_2025 = calculate_cfcs_score(monthly_corr_2025)

print("=== 2025 CFCS SCORE ===")
for k, v in score_2025.items():
    print(f"{k}: {v}")

=== 2025 CFCS SCORE ===
cfcs_score: 71.56
avg_significant_correlation: 0.7419
max_abs_correlation: 0.9995
significant_correlations_pct: 22.37
total_correlations: 225930
significant_correlations: 50548


In [56]:
def build_feature_aggregates(
    correlations_df: pd.DataFrame,
    sig_thr: float,
) -> dict[str, dict]:
    stats = {}

    for f, g in correlations_df.groupby("climate_variable"):
        abs_corr = g["correlation"].abs().to_numpy()
        sig = abs_corr >= sig_thr

        stats[f] = {
            "n_total": abs_corr.size,
            "n_sig": int(sig.sum()),
            "sum_sig_abs": float(abs_corr[sig].sum()),
            "max_abs": float(abs_corr.max()),
        }

    return stats

def calculate_cfcs_from_stats(
    feature_stats: dict[str, dict],
    features: list[str],
    sig_thr: float,
) -> dict:
    n_total = n_sig = 0
    sum_sig = max_abs = 0.0

    for f in features:
        s = feature_stats.get(f)
        if s is None:
            continue
        n_total += s["n_total"]
        n_sig += s["n_sig"]
        sum_sig += s["sum_sig_abs"]
        max_abs = max(max_abs, s["max_abs"])

    abs_corr = np.empty(0) if n_total == 0 else None

    if n_total == 0:
        return calculate_cfcs_from_arrays(abs_corr, sig_thr)

    avg_sig = sum_sig / n_sig if n_sig > 0 else 0.0
    sig_pct = n_sig / n_total

    cfcs = (
        0.5 * min(100.0, avg_sig * 100)
        + 0.3 * min(100.0, max_abs * 100)
        + 0.2 * (sig_pct * 100)
    )

    return {
        "cfcs_score": round(cfcs, 2),
        "avg_significant_correlation": round(avg_sig, 4),
        "max_abs_correlation": round(max_abs, 4),
        "significant_correlations_pct": round(sig_pct * 100, 2),
        "total_correlations": int(n_total),
        "significant_correlations": int(n_sig),
    }

## Feature Selection with Beam Search

In [57]:
import heapq

def beam_cfcs_selection(
    correlations_df: pd.DataFrame,
    sig_thr: float = 0.5,
    beam_width: int = 20,
    max_features: int = 8,
) -> tuple[list[str], dict]:
    corr_df = correlations_df[
        ["climate_variable", "correlation"]
    ].dropna()

    stats = build_feature_aggregates(corr_df, sig_thr)
    features = list(stats)

    def score(feats: list[str]) -> float:
        return calculate_cfcs_from_stats(
            stats,
            feats,
            sig_thr,
        )["cfcs_score"]

    # (score, feature_list)
    beam: list[tuple[float, list[str]]] = [(0.0, [])]
    best: tuple[float, list[str]] = (0.0, [])

    for _ in range(max_features):
        candidates: list[tuple[float, list[str]]] = []

        for _, feats in beam:
            used = set(feats)
            for f in features:
                if f in used:
                    continue

                new_feats = feats + [f]
                s = score(new_feats)

                candidates.append((s, new_feats))

        # Keep top beam_width candidates
        beam = heapq.nlargest(
            beam_width,
            candidates,
            key=lambda x: x[0],
        )

        if not beam:
            break

        if beam[0][0] > best[0]:
            best = beam[0]

    best_stats = calculate_cfcs_from_stats(
        stats,
        best[1],
        sig_thr,
    )

    return best[1], best_stats

In [58]:
beam_selected_features, beam_cfcs_stats = beam_cfcs_selection(
    correlations_df=monthly_corr_2025,
    sig_thr=0.5,
    beam_width=50,
    max_features=30,
)
print(beam_cfcs_stats)
print()
print(beam_selected_features)

{'cfcs_score': 85.02, 'avg_significant_correlation': 0.8412, 'max_abs_correlation': 0.9976, 'significant_correlations_pct': 65.15, 'total_correlations': 2244, 'significant_correlations': 1462}

['climate_risk_drought_weighted_country_cumsum']


## Holdout Validation with Selected Features

In [59]:
selected_df_2025 = (
    df_engineered[df_engineered["date_on"].dt.year == 2025]
    .dropna()
    [BASE_COLS + beam_selected_features + FUTURES_COLS] 
)

selected_monthly_corr_2025 = compute_monthly_climate_futures_correlations(
    selected_df_2025
)

selected_score_2025 = calculate_cfcs_score(selected_monthly_corr_2025)

print("=== 2025 CFCS SCORE (Selected Feature/Features) ===")
for k, v in selected_score_2025.items():
    print(f"{k}: {v}")

=== 2025 CFCS SCORE (Selected Feature/Features) ===
cfcs_score: 81.83
avg_significant_correlation: 0.7856
max_abs_correlation: 0.9984
significant_correlations_pct: 63.0
total_correlations: 2057
significant_correlations: 1296


## Let's Submit the Solution

In [29]:
valid_idx = df_engineered.dropna().index
df_submit = df_engineered.loc[
    valid_idx,
    BASE_COLS + beam_selected_features + FUTURES_COLS,
]

output_path = (
    DATA_DIR
    / "submissions"
    / "submission.csv"
)
df_submit.to_csv(output_path, index=False)
print(f"Saved to {output_path}")
print(f"Rows: {len(df_submit)}, Columns: {len(df_submit.columns)}")

OSError: Cannot save file into a non-existent directory: '/home/chhayly-sreng/helios/helios-kaggle-competition/data/submissions'